In [ ]:
# from base import fetch_dataset
# from grakel.datasets import fetch_dataset
import multiprocessing
# multiprocessing.set_start_method('forkserver')
from concurrent.futures import ProcessPoolExecutor
import controlpy
import time
import networkx as nx
import numpy as np
import csv
import itertools
import scipy
import numpy.linalg as LA
import scipy as sc
from scipy import stats
import numpy as np
from os import listdir
from os.path import join
# from igraph import  *
import scipy as sc
from scipy import stats
import numpy as np
import pandas as pd
# import sys,igraph
import scipy.spatial.distance
from math import sqrt
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import *
from scipy import sparse
import sys
## from joblib import Parallel, delayed
from joblib import Parallel, delayed
import multiprocessing
from tqdm.notebook import tqdm
import tqdm
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from torch_geometric.datasets import TUDataset
from sklearn.utils import shuffle
num_cores = multiprocessing.cpu_count()

In [ ]:
#from grakel.datasets import fetch_dataset
#default parameters
INCLUDE_GRM = True
INCLUDE_INVERSE_GRM = False #-----
INCLUDE_EIGEN_VALUES = True
INCLUDE_MIN_MAX = True
INCLUDE_MEAN = True
INCLUDE_LAP_SPECT = True
INCLUDE_LAP_SPECT_EXT = True
INCLUDE_METRIC_DIMENSION = True
INCLUDE_FEATURES = True
LAPLACIAN_ENERGY = True
ECC_SPECTRUM = True
ECC_ENERGY = True
WEINER_INDEX = True
TRACE_DEG_SEQ = True
INCLUDE_CYCLES = True

def get_descriptor(G,num_iterations = 30):
#     G = G.subgraph(max(nx.connected_components(G), key=len) )
    G = nx.convert_node_labels_to_integers(G)
    if G.number_of_nodes()<10:
#         return list(np.zeros(162, dtype = float))
        G = add_multiple_copies(G)
    A1 = -(np.matrix(nx.laplacian_matrix(G,nodelist=sorted(G.nodes())).todense()))
    n = A1.shape[0]
    desc = {}
    leaders = []
    for num_l_idx, num_leaders in enumerate([1,2,5,9,int(n*2/100)+1,int(n*5/100)+1,int(n*10/100)+1,int(n*20/100)+1,int(n*30/100)+1]):
#         A1 = np.matrix( nx.laplacian_matrix(G).todense() )
        old_n = A1.shape[0]
        traces = []
        ranks = []
        min_eigs = []
        max_eigs = []
        itraces = []
        iranks = []
        imin_eigs = []
        imax_eigs = []
        metric_dimension = []
        for i in range(num_iterations):
            follows = np.random.choice(old_n,old_n-num_leaders,replace=False)
            leaders = list(set(range(old_n))-set(follows))
            A2 = A1[follows, :]
            A3 = A2[:, follows]
            A = 1*A3
            n = A.shape[0]
            mask = np.ones(old_n, dtype=bool)
            mask[leaders] = False
            B = A1[mask,:] #removing corresponding rows of leaders from laplacian
            B = B[:,leaders] #selecting corresponding columns of leaders from laplacian
            A = np.mat(A)
            B = np.mat(B)
            if INCLUDE_GRM:
                grm = controlpy.analysis.controllability_gramian(A,B)
                rnk = LA.matrix_rank(grm)
                ranks.append(rnk)
                traces.append( np.trace(grm) )
                if INCLUDE_EIGEN_VALUES :
                    w, v = LA.eig(grm)
                    a = np.real(w)
                    a[a == 0] = 0.0001
                    minval = np.min(a)
                    min_eigs.append( minval ) 
                    max_eigs.append(np.max( a ) )
                if INCLUDE_INVERSE_GRM :
                    try:
                        grm = LA.pinv( grm ,hermitian=True)
                    except:
                        print("Oops!", sys.exc_info()[0], "occurred.")
                        return zero_des()
                    itraces.append(np.trace(grm))
                    iranks.append( LA.matrix_rank(grm) )
                    if INCLUDE_EIGEN_VALUES :
                        w, v = LA.eig(grm)
                        a = np.real(w)
                        a[a == 0] = 0.0001
                        #if sum(a) == 0:
                        #    print(a)
                        #    print(grm)
                        #    print(A1)
                        minval = np.min(a)
                        imin_eigs.append( np.min( minval ) )
                        imax_eigs.append( np.max( a ) )  

            if INCLUDE_METRIC_DIMENSION:            
                metric_dimension.append(compute_metric_dimension(G, leaders))

        if INCLUDE_MIN_MAX :
            desc['GRM_MAX_TRACE_'+str(num_l_idx)] = np.max(traces)
            desc['GRM_MIN_TRACE_'+str(num_l_idx)] = np.min(traces) 
            desc['GRM_MAX_RANK_'+str(num_l_idx)]  = np.max(ranks) 
            desc['GRM_MIN_RANK_'+str(num_l_idx)]  = np.min(ranks) 
            if INCLUDE_EIGEN_VALUES :

                desc['GRM_MAX_of_MIN_EIG_'+str(num_l_idx)] = np.max(min_eigs)
                desc['GRM_MIN_of_MIN_EIG_'+str(num_l_idx)] = np.min(min_eigs)

                desc['GRM_MAX_of_MAX_EIG_'+str(num_l_idx)] = np.max(max_eigs)
                desc['GRM_MIN_of_MAX_EIG_'+str(num_l_idx)] = np.min(max_eigs)
            if INCLUDE_INVERSE_GRM :    

                desc['INV_GRM_MAX_TRACE_'+str(num_l_idx)] = np.max(itraces)
                desc['INV_GRM_MIN_TRACE_'+str(num_l_idx)] = np.min(itraces)

        if INCLUDE_MEAN :
            desc['GRM_MEAN_TRACE_'+str(num_l_idx)] = np.mean(traces)
            desc['GRM_MEAN_RANK_'+str(num_l_idx)] = np.mean(ranks)

            if INCLUDE_EIGEN_VALUES :
                desc['GRM_MEAN_MAX_EIG_'+str(num_l_idx)] = np.mean(max_eigs)
                desc['GRM_MEAN_MIN_EIG_'+str(num_l_idx)] = np.mean(min_eigs)

            if INCLUDE_INVERSE_GRM :
                desc['INV_GRM_MEAN_TRACE_'+str(num_l_idx)] = np.mean(itraces)
                desc['INV_GRM_MEAN_RANK_'+str(num_l_idx)] = np.mean(iranks)

                if INCLUDE_EIGEN_VALUES :
                    desc['INV_GRM_MEAN_MIN_EIG_'+str(num_l_idx)] = np.mean(imin_eigs)
                    desc['INV_GRM_MEAN_MAX_EIG_'+str(num_l_idx)] =np.mean(imax_eigs)

        if INCLUDE_METRIC_DIMENSION:
            desc['METRIC_DIMENSION_MEAN'+str(num_l_idx)] = np.mean(metric_dimension)
            desc['METRIC_DIMENSION_MIN'+str(num_l_idx)] = np.min(metric_dimension)
            desc['METRIC_DIMENSION_MAX'+str(num_l_idx)] = np.max(metric_dimension)

    n,m = G.number_of_nodes(),G.number_of_edges()
    desc['no_nodes'] = n
    desc['no_edges'] = m
    desc['no_bi_conn_comp'] =len(list(nx.biconnected_components(G)))
    if INCLUDE_LAP_SPECT :
        Ls = sorted(nx.laplacian_spectrum(G))
        desc['LS_0'] = Ls[0]
        desc['LS_1'] = Ls[1]
        desc['LS_2'] = Ls[2]
    if INCLUDE_LAP_SPECT_EXT :
        desc['LSE_3'] = Ls[3]
        desc['LSE_4'] = Ls[4]
        desc['LSE_-1'] = Ls[-1]
        desc['LSE_-2'] = Ls[-2]
        desc['LSE_-3'] = Ls[-3]
    if INCLUDE_CYCLES:
        if n-m == 1:
            desc['0cyc'] = 1  
        else:
            desc['0cyc'] = 0
        if n-m == 0:
            desc['1cyc'] = 1
        else:
            desc['1cyc'] = 0
        if n-m == -1:
            desc['2cyc'] = 1
        else:
            desc['2cyc'] = 0
        if n-m < -1:
            desc['g2cyc'] = 1
        else:
            desc['g2cyc'] = 0

    if INCLUDE_FEATURES:
        desc.update(add_features(G))
    return(desc)
    

def add_multiple_copies(G):
    no_node = G.number_of_nodes()   
    H = G.copy()
    H = nx.convert_node_labels_to_integers(H, ordering='sorted')
    val = sorted(G.nodes())
    val0 = val[0]
    while H.number_of_nodes()<10:
        val1 = H.number_of_nodes()-1
        H = nx.union(H, G, rename=('G1-', 'G2-'))
        H.add_edge('G1-'+str(val1), 'G2-'+str(val0))
        H = nx.convert_node_labels_to_integers(H, ordering='sorted')
    return H

def trace_deq_seq(G):
    seq = sorted(list(dict(nx.degree(G)).values()),reverse = False)
    trc = 0
    for idx, i in enumerate(seq):
        if idx+1<=i:
            trc = idx+1
    return trc

def add_features(G):
    L = nx.laplacian_matrix(G).todense()
    eig = LA.eigvals(L)
    avg_deg = 2*(G.number_of_edges()/G.number_of_nodes()) # made it faster
    lap_energy = sum([abs(i-avg_deg) for i in eig])
    dist_matrix =np.array(nx.floyd_warshall_numpy(G,nodelist=sorted(G.nodes()))) #nodelist, important argument, I will let you know
    eccentricity = dist_matrix * (dist_matrix >= np.sort(dist_matrix, axis=1)[:,[-1]]).astype(int) #can be computed through networkx thats why nodelist argument is added
    e_vals = LA.eigvals(eccentricity) 
    largest_eig = np.real(max(e_vals))
    energy = np.real(sum([abs(x) for x in e_vals]))
    wiener_index = nx.wiener_index(G)
    trace_DS = trace_deq_seq(G)  #it was different then your implementation kindly read the document again as sent by the dr. Waseem or go through by my implementation
    return {'lap_energy':lap_energy,'ecc_spectrum':largest_eig,'ecc_energy':energy,'wiener_index':wiener_index,'trace_deg_seq':trace_DS}

def compute_metric_dimension(G, leaders):
    nodes_list = list(G.nodes()) 
    distance_vec = []
    for n in nodes_list:
        v_ = [nx.shortest_path_length(G, source=n, target=l) for l in leaders]
        distance_vec.append(v_)
    distance_vec = np.array(distance_vec)
    return np.unique(distance_vec, axis=0).shape[0]

        
def add_vertex(G):
    G = nx.convert_node_labels_to_integers(G)
    nodes = sorted(list(G.nodes()))
    new_vertex = nodes[-1]+1 
    G.add_node(new_vertex)
    for n in G.nodes():
        if n !=new_vertex:
            G.add_edge(new_vertex, n)
    return G
    
def zero_des(): 
    return {'GRM_MAX_TRACE_0': 0.0, 'GRM_MIN_TRACE_0': 0.0, 'GRM_MAX_RANK_0': 0.0, 'GRM_MIN_RANK_0': 0.0, 'GRM_MAX_of_MIN_EIG_0': 0.0, 'GRM_MIN_of_MIN_EIG_0': 0.0, 'GRM_MAX_of_MAX_EIG_0': 0.0, 'GRM_MIN_of_MAX_EIG_0': 0.0, 'INV_GRM_MAX_TRACE_0': 0.0, 'INV_GRM_MIN_TRACE_0': 0.0, 'GRM_MEAN_TRACE_0': 0.0, 'GRM_MEAN_RANK_0': 0.0, 'GRM_MEAN_MAX_EIG_0': 0.0, 'GRM_MEAN_MIN_EIG_0': 0.0, 'INV_GRM_MEAN_TRACE_0': 0.0, 'INV_GRM_MEAN_RANK_0': 0.0, 'INV_GRM_MEAN_MIN_EIG_0': 0.0, 'INV_GRM_MEAN_MAX_EIG_0': 0.0, 'METRIC_DIMENSION_MEAN0': 0.0, 'METRIC_DIMENSION_MIN0': 0.0, 'METRIC_DIMENSION_MAX0': 0.0, 'GRM_MAX_TRACE_1': 0.0, 'GRM_MIN_TRACE_1': 0.0, 'GRM_MAX_RANK_1': 0.0, 'GRM_MIN_RANK_1': 0.0, 'GRM_MAX_of_MIN_EIG_1': 0.0, 'GRM_MIN_of_MIN_EIG_1': 0.0, 'GRM_MAX_of_MAX_EIG_1': 0.0, 'GRM_MIN_of_MAX_EIG_1': 0.0, 'INV_GRM_MAX_TRACE_1': 0.0, 'INV_GRM_MIN_TRACE_1': 0.0, 'GRM_MEAN_TRACE_1': 0.0, 'GRM_MEAN_RANK_1': 0.0, 'GRM_MEAN_MAX_EIG_1': 0.0, 'GRM_MEAN_MIN_EIG_1': 0.0, 'INV_GRM_MEAN_TRACE_1': 0.0, 'INV_GRM_MEAN_RANK_1': 0.0, 'INV_GRM_MEAN_MIN_EIG_1': 0.0, 'INV_GRM_MEAN_MAX_EIG_1': 0.0, 'METRIC_DIMENSION_MEAN1': 0.0, 'METRIC_DIMENSION_MIN1': 0.0, 'METRIC_DIMENSION_MAX1': 0.0, 'GRM_MAX_TRACE_2': 0.0, 'GRM_MIN_TRACE_2': 0.0, 'GRM_MAX_RANK_2': 0.0, 'GRM_MIN_RANK_2': 0.0, 'GRM_MAX_of_MIN_EIG_2': 0.0, 'GRM_MIN_of_MIN_EIG_2': 0.0, 'GRM_MAX_of_MAX_EIG_2': 0.0, 'GRM_MIN_of_MAX_EIG_2': 0.0, 'INV_GRM_MAX_TRACE_2': 0.0, 'INV_GRM_MIN_TRACE_2': 0.0, 'GRM_MEAN_TRACE_2': 0.0, 'GRM_MEAN_RANK_2': 0.0, 'GRM_MEAN_MAX_EIG_2': 0.0, 'GRM_MEAN_MIN_EIG_2': 0.0, 'INV_GRM_MEAN_TRACE_2': 0.0, 'INV_GRM_MEAN_RANK_2': 0.0, 'INV_GRM_MEAN_MIN_EIG_2': 0.0, 'INV_GRM_MEAN_MAX_EIG_2': 0.0, 'METRIC_DIMENSION_MEAN2': 0.0, 'METRIC_DIMENSION_MIN2': 0.0, 'METRIC_DIMENSION_MAX2': 0.0, 'GRM_MAX_TRACE_3': 0.0, 'GRM_MIN_TRACE_3': 0.0, 'GRM_MAX_RANK_3': 0.0, 'GRM_MIN_RANK_3': 0.0, 'GRM_MAX_of_MIN_EIG_3': 0.0, 'GRM_MIN_of_MIN_EIG_3': 0.0, 'GRM_MAX_of_MAX_EIG_3': 0.0, 'GRM_MIN_of_MAX_EIG_3': 0.0, 'INV_GRM_MAX_TRACE_3': 0.0, 'INV_GRM_MIN_TRACE_3': 0.0, 'GRM_MEAN_TRACE_3': 0.0, 'GRM_MEAN_RANK_3': 0.0, 'GRM_MEAN_MAX_EIG_3': 0.0, 'GRM_MEAN_MIN_EIG_3': 0.0, 'INV_GRM_MEAN_TRACE_3': 0.0, 'INV_GRM_MEAN_RANK_3': 0.0, 'INV_GRM_MEAN_MIN_EIG_3': 0.0, 'INV_GRM_MEAN_MAX_EIG_3': 0.0, 'METRIC_DIMENSION_MEAN3': 0.0, 'METRIC_DIMENSION_MIN3': 0.0, 'METRIC_DIMENSION_MAX3': 0.0, 'GRM_MAX_TRACE_4': 0.0, 'GRM_MIN_TRACE_4': 0.0, 'GRM_MAX_RANK_4': 0.0, 'GRM_MIN_RANK_4': 0.0, 'GRM_MAX_of_MIN_EIG_4': 0.0, 'GRM_MIN_of_MIN_EIG_4': 0.0, 'GRM_MAX_of_MAX_EIG_4': 0.0, 'GRM_MIN_of_MAX_EIG_4': 0.0, 'INV_GRM_MAX_TRACE_4': 0.0, 'INV_GRM_MIN_TRACE_4': 0.0, 'GRM_MEAN_TRACE_4': 0.0, 'GRM_MEAN_RANK_4': 0.0, 'GRM_MEAN_MAX_EIG_4': 0.0, 'GRM_MEAN_MIN_EIG_4': 0.0, 'INV_GRM_MEAN_TRACE_4': 0.0, 'INV_GRM_MEAN_RANK_4': 0.0, 'INV_GRM_MEAN_MIN_EIG_4': 0.0, 'INV_GRM_MEAN_MAX_EIG_4': 0.0, 'METRIC_DIMENSION_MEAN4': 0.0, 'METRIC_DIMENSION_MIN4': 0.0, 'METRIC_DIMENSION_MAX4': 0.0, 'GRM_MAX_TRACE_5': 0.0, 'GRM_MIN_TRACE_5': 0.0, 'GRM_MAX_RANK_5': 0.0, 'GRM_MIN_RANK_5': 0.0, 'GRM_MAX_of_MIN_EIG_5': 0.0, 'GRM_MIN_of_MIN_EIG_5': 0.0, 'GRM_MAX_of_MAX_EIG_5': 0.0, 'GRM_MIN_of_MAX_EIG_5': 0.0, 'INV_GRM_MAX_TRACE_5': 0.0, 'INV_GRM_MIN_TRACE_5': 0.0, 'GRM_MEAN_TRACE_5': 0.0, 'GRM_MEAN_RANK_5': 0.0, 'GRM_MEAN_MAX_EIG_5': 0.0, 'GRM_MEAN_MIN_EIG_5': 0.0, 'INV_GRM_MEAN_TRACE_5': 0.0, 'INV_GRM_MEAN_RANK_5': 0.0, 'INV_GRM_MEAN_MIN_EIG_5': 0.0, 'INV_GRM_MEAN_MAX_EIG_5': 0.0, 'METRIC_DIMENSION_MEAN5': 0.0, 'METRIC_DIMENSION_MIN5': 0.0, 'METRIC_DIMENSION_MAX5': 0.0, 'GRM_MAX_TRACE_6': 0.0, 'GRM_MIN_TRACE_6': 0.0, 'GRM_MAX_RANK_6': 0.0, 'GRM_MIN_RANK_6': 0.0, 'GRM_MAX_of_MIN_EIG_6': 0.0, 'GRM_MIN_of_MIN_EIG_6': 0.0, 'GRM_MAX_of_MAX_EIG_6': 0.0, 'GRM_MIN_of_MAX_EIG_6': 0.0, 'INV_GRM_MAX_TRACE_6': 0.0, 'INV_GRM_MIN_TRACE_6': 0.0, 'GRM_MEAN_TRACE_6': 0.0, 'GRM_MEAN_RANK_6': 0.0, 'GRM_MEAN_MAX_EIG_6': 0.0, 'GRM_MEAN_MIN_EIG_6': 0.0, 'INV_GRM_MEAN_TRACE_6': 0.0, 'INV_GRM_MEAN_RANK_6': 0.0, 'INV_GRM_MEAN_MIN_EIG_6': 0.0, 'INV_GRM_MEAN_MAX_EIG_6': 0.0, 'METRIC_DIMENSION_MEAN6': 0.0, 'METRIC_DIMENSION_MIN6': 0.0, 'METRIC_DIMENSION_MAX6': 0.0, 'GRM_MAX_TRACE_7': 0.0, 'GRM_MIN_TRACE_7': 0.0, 'GRM_MAX_RANK_7': 0.0, 'GRM_MIN_RANK_7': 0.0, 'GRM_MAX_of_MIN_EIG_7': 0.0, 'GRM_MIN_of_MIN_EIG_7': 0.0, 'GRM_MAX_of_MAX_EIG_7': 0.0, 'GRM_MIN_of_MAX_EIG_7': 0.0, 'INV_GRM_MAX_TRACE_7': 0.0, 'INV_GRM_MIN_TRACE_7': 0.0, 'GRM_MEAN_TRACE_7': 0.0, 'GRM_MEAN_RANK_7': 0.0, 'GRM_MEAN_MAX_EIG_7': 0.0, 'GRM_MEAN_MIN_EIG_7': 0.0, 'INV_GRM_MEAN_TRACE_7': 0.0, 'INV_GRM_MEAN_RANK_7': 0.0, 'INV_GRM_MEAN_MIN_EIG_7': 0.0, 'INV_GRM_MEAN_MAX_EIG_7': 0.0, 'METRIC_DIMENSION_MEAN7': 0.0, 'METRIC_DIMENSION_MIN7': 0.0, 'METRIC_DIMENSION_MAX7': 0.0, 'GRM_MAX_TRACE_8': 0.0, 'GRM_MIN_TRACE_8': 0.0, 'GRM_MAX_RANK_8': 0.0, 'GRM_MIN_RANK_8': 0.0, 'GRM_MAX_of_MIN_EIG_8': 0.0, 'GRM_MIN_of_MIN_EIG_8': 0.0, 'GRM_MAX_of_MAX_EIG_8': 0.0, 'GRM_MIN_of_MAX_EIG_8': 0.0, 'INV_GRM_MAX_TRACE_8': 0.0, 'INV_GRM_MIN_TRACE_8': 0.0, 'GRM_MEAN_TRACE_8': 0.0, 'GRM_MEAN_RANK_8': 0.0, 'GRM_MEAN_MAX_EIG_8': 0.0, 'GRM_MEAN_MIN_EIG_8': 0.0, 'INV_GRM_MEAN_TRACE_8': 0.0, 'INV_GRM_MEAN_RANK_8': 0.0, 'INV_GRM_MEAN_MIN_EIG_8': 0.0, 'INV_GRM_MEAN_MAX_EIG_8': 0.0, 'METRIC_DIMENSION_MEAN8': 0.0, 'METRIC_DIMENSION_MIN8': 0.0, 'METRIC_DIMENSION_MAX8': 0.0, 'no_nodes': 0.0, 'no_edges': 0.0, 'no_bi_conn_comp': 0.0, 'LS_0': 0.0, 'LS_1': 0.0, 'LS_2': 0.0, 'LSE_3': 0.0, 'LSE_4': 0.0, 'LSE_-1': 0.0, 'LSE_-2': 0.0, 'LSE_-3': 0.0, '0cyc': 0.0, '1cyc': 0.0, '2cyc': 0.0, 'g2cyc': 0.0, 'lap_energy': 0.0, 'ecc_spectrum': 0.0, 'ecc_energy': 0.0, 'wiener_index': 0.0, 'trace_deg_seq': 0.0}
def return_dataset(file_name):
    #i = 'G_nci1'
    dd = fetch_dataset(file_name)
    graph_list = []
    for gg in dd.data:
        g_ = nx.Graph()
        g_.add_edges_from([(i[0], i[1]) for i in gg[0]])
        graph_list.append(g_)
    data_y = dd.target
    return graph_list, data_y

def apply_RF_Grid(data_X,data_y):
    num_cores = multiprocessing.cpu_count()
    max_arr = []
    estimator  = RandomForestClassifier(criterion='gini', max_depth=None, min_weight_fraction_leaf=0.0,
                                            max_leaf_nodes=None, bootstrap=True, 
                                            oob_score=False, n_jobs=num_cores,verbose=0, warm_start=False,
                                            class_weight=None)
         
    param_grid = {'n_estimators':[50,100,500], 'max_features':['sqrt'], 
                  'min_samples_split':[2,3,4,5,10], 'min_samples_leaf':[1,2,5]}
    kf = StratifiedKFold(n_splits=10, random_state = 890, shuffle = True)
    grid_rf    = GridSearchCV(estimator, param_grid, scoring='accuracy', n_jobs=num_cores, 
                 refit=True, cv=kf, verbose=1, pre_dispatch='n_jobs', 
                 error_score='raise')

    grid_rf.fit(data_X, data_y)
    
    cv_results = pd.DataFrame(grid_rf.cv_results_)
    std = cv_results.loc[grid_rf.best_index_].std_test_score
    best_score = grid_rf.best_score_
    return best_score



In [ ]:
if __name__ == '__main__':
    datasets = ["PROTEINS_full", "DD","IMDB-BINARY","IMDB-MULTI","REDDIT-BINARY","REDDIT-MULTI-5K"]
    folds = 3
    file = open("results/social_networks.csv",'a',newline = '')
    res_writer = csv.writer(file, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    res_writer.writerow(["dataset","CTRL_ACC","std","time"])
    for d in datasets:
        graph_dataset = TUDataset(root = "data/TUDataset", name=d)
        graphs = [to_networkx(d, to_undirected = True) for d in graph_dataset]
        graph_data      = [G if nx.is_connected(G) else add_vertex(G) for G in graphs]

        print("dataset {} is loaded with {} number of graphs".format(d,len(graph_data)))
        accuracy = []
        for i in range(folds):
            start = time.time()
            control_des = Parallel(n_jobs=-1, verbose=2)(delayed(get_descriptor)
                                (G) for G in graph_data)
            labels = [int(x.y.item()) for x in graph_dataset]
            end = time.time()
            print("time:",end-start)
            print("dataset {} loaded successfully,control desciptor shape:{}".format(d, len(control_des)))
            control_des = pd.DataFrame(control_des)
            acc = apply_RF_Grid(control_des, labels)
            print(acc)
            accuracy.append(acc)
        to_write = [d,np.mean(acc), np.std(accuracy)]
        res_writer.writerow(to_write)
        print("features of {} dataset saved successfully! acc, std, time: {}, {}".format(d,np.mean(acc)*100, np.std(accuracy)*100))
        file.flush()
#         break
    file.close()    